In [1]:
import $ivy.`org.apache.spark::spark-sql:3.1.2`

import $ivy.$                                  

In [2]:
val path = java.nio.file.FileSystems.getDefault().getPath("/home/yuanli/work/pcbs/re-build/spark-rapids/dist/target/rapids-4-spark_2.12-22.06.0-SNAPSHOT-cuda11.jar")
val x = ammonite.ops.Path(path)
interp.load.cp(x)

path: java.nio.file.Path = /home/yuanli/work/pcbs/re-build/spark-rapids/dist/target/rapids-4-spark_2.12-22.06.0-SNAPSHOT-cuda11.jar
x: os.Path = root/'home/'yuanli/'work/'pcbs/"re-build"/"spark-rapids"/'dist/'target/"rapids-4-spark_2.12-22.06.0-SNAPSHOT-cuda11.jar"

In [3]:
import $ivy.`org.apache.spark::spark-avro:3.1.2`

import $ivy.$                                   

In [1]:
import java.lang.reflect.Method

import org.apache.spark.{sql, TaskContext}
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.catalyst.InternalRow
import org.apache.spark.sql.catalyst.expressions.Attribute
import org.apache.spark.sql.columnar.{CachedBatch, CachedBatchSerializer}
import org.apache.spark.sql.execution.columnar.DefaultCachedBatchSerializer
import org.apache.spark.sql.execution.SparkPlan
import org.apache.spark.sql.internal.SQLConf
import org.apache.spark.sql.vectorized.ColumnarBatch
import org.apache.spark.storage.StorageLevel
import org.apache.spark.storage.StorageLevel.MEMORY_ONLY
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.execution.columnar.DefaultCachedBatch
import org.apache.spark.sql.rapids.GpuInMemoryTableScanExec
import com.nvidia.spark.GpuCachedBatchSerializer
import com.nvidia.spark.rapids.ParquetCachedBatch
import org.apache.spark.sql.catalyst.plans.logical.LogicalPlan
import org.apache.spark.sql.execution.columnar.InMemoryRelation
import com.nvidia.spark.ParquetCachedBatchSerializer

import scala.reflect.runtime.universe._


In [2]:
val RAPIDS_JAR = "/home/yuanli/work/pcbs/re-build/spark-rapids/dist/target/rapids-4-spark_2.12-22.06.0-SNAPSHOT-cuda11.jar"

def createSparkSession(): SparkSession = {
    val conf = new SparkConf()
    conf.setMaster("spark://yuanli-System-Product-Name:7077")
    conf.setAppName("PCBS Benchmark")
    conf.set("spark.jars", "/home/yuanli/work/pcbs/re-build/spark-rapids/dist/target/rapids-4-spark_2.12-22.06.0-SNAPSHOT-cuda11.jar")
    conf.set("spark.driver.memory", "4G")
    conf.set("spark.executor.memory", "4G")
    conf.set("spark.executor.cores", "4")
    conf.set("spark.locality.wait", "0")
    conf.set("spark.dynamicAllocation.enabled", "false")

    //rapids config
    conf.set("spark.rapids.sql.enabled", "true") 
    conf.set("spark.plugins", "com.nvidia.spark.SQLPlugin")
    conf.set("spark.executor.resource.gpu.amount", "1")
    conf.set("spark.task.resource.gpu.amount", "0.25") 
    conf.set("spark.rapids.sql.concurrentGpuTasks", "2")
    conf.set("spark.rapids.memory.pinnedPool.size", "1G")
    
    conf.set("spark.driver.extraJavaOptions","-ea -Duser.timezone=UTC")
    conf.set("spark.executor.extraJavaOptions","-ea -Duser.timezone=UTC -Dai.rapids.cudf.prefer-pinned=true")
    conf.set("spark.sql.cache.serializer","com.nvidia.spark.ParquetCachedBatchSerializer")
    conf.set("spark.driver.extraClassPath", RAPIDS_JAR)
    conf.set("spark.executor.extraClassPath", RAPIDS_JAR)

    SparkSession.builder.config(conf=conf).getOrCreate()

  }

RAPIDS_JAR = /home/yuanli/work/pcbs/re-build/spark-rapids/dist/target/rapids-4-spark_2.12-22.06.0-SNAPSHOT-cuda11.jar


createSparkSession: ()org.apache.spark.sql.SparkSession


/home/yuanli/work/pcbs/re-build/spark-rapids/dist/target/rapids-4-spark_2.12-22.06.0-SNAPSHOT-cuda11.jar

In [3]:
import com.nvidia.spark.ParquetCachedBatchSerializer

import java.lang.reflect.Method
import org.apache.spark.{sql, TaskContext}
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.catalyst.InternalRow
import org.apache.spark.sql.catalyst.expressions.Attribute
import org.apache.spark.sql.columnar.{CachedBatch, CachedBatchSerializer}
import org.apache.spark.sql.execution.SparkPlan
import org.apache.spark.sql.internal.SQLConf

import org.apache.spark.sql.vectorized.ColumnarBatch
import org.apache.spark.storage.StorageLevel
import org.apache.spark.storage.StorageLevel.MEMORY_ONLY

import java.io.{BufferedWriter, File, FileWriter, IOException}


// // import org.apache.spark.sql.rapids.GpuInMemoryTableScanExec
import org.apache.spark.sql.rapids._


val file = new File("/home/yuanli/work/reviews/raza-pcbs-pr159/spark-rapids-examples/examples/pcbs-benchmark/aaaaa.txt")
val bw = new BufferedWriter(new FileWriter(file, true))


file = /home/yuanli/work/reviews/raza-p...


/home/yuanli/work/reviews/raza-p...

In [4]:

class TestCachedBatchSerializer(
    useCompression: Boolean,
    batchSize: Int) extends DefaultCachedBatchSerializer {

  override def convertInternalRowToCachedBatch(input: RDD[InternalRow],
      schema: Seq[Attribute],
      storageLevel: StorageLevel,
      conf: SQLConf): RDD[CachedBatch] = {
    convertForCacheInternal(input, schema, batchSize, useCompression)
  }
}

defined class TestCachedBatchSerializer


In [5]:
case class CloseableColumnBatchIterator(iter: Iterator[ColumnarBatch]) extends
    Iterator[ColumnarBatch] {
  var cb: ColumnarBatch = _

  private def closeCurrentBatch(): Unit = {
    if (cb != null) {
      cb.close()
      cb = null
    }
  }

  TaskContext.get().addTaskCompletionListener[Unit]((_: TaskContext) => {
    closeCurrentBatch()
  })

  override def hasNext: Boolean = iter.hasNext

  override def next(): ColumnarBatch = {
    closeCurrentBatch()
    cb = iter.next()
    cb
  }
}


defined class CloseableColumnBatchIterator


In [6]:
def callPrivate(obj: AnyRef, methodName: String, parameters:AnyRef*) = {
  val parameterTypes = parameters.map(_.getClass())
  val method = obj.getClass.getDeclaredMethod(methodName, parameterTypes:_*)
  method.setAccessible(true)
  method.invoke(obj, parameters:_*)
}

callPrivate: (obj: AnyRef, methodName: String, parameters: AnyRef*)Object


In [7]:
import scala.reflect.runtime.universe._
def getReflectLogicalPlan(df: DataFrame) : LogicalPlan = {
    val mirror = runtimeMirror(scala.reflect.runtime.universe.getClass.getClassLoader)
    val instanceMirror = mirror.reflect(df)
    val field = scala.reflect.runtime.universe.typeOf[DataFrame].decl(TermName("logicalPlan")).asTerm
    val fieldMirror = instanceMirror.reflectField(field)
    fieldMirror.get.asInstanceOf[LogicalPlan]
}

getReflectLogicalPlan: (df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.catalyst.plans.logical.LogicalPlan


In [8]:

def readWriteCache(
      acc: Boolean,
      spark: SparkSession,
      ser: CachedBatchSerializer,
      func: DataFrame => (Method, Seq[SparkPlan]),
      verifyFunc: CachedBatch => Any,
      query: SparkSession => sql.DataFrame) = {
//     CachePerfWriter.appendLine("Writing cache 5 times")
    val writes = for (_ <- 0 until 5) yield {
        
      val df = query(spark).cache()
      val storageLevel = MEMORY_ONLY
//       val logicplan = df.logicalPlan
    //try 1:  java.lang.ClassCastException: java.lang.reflect.Field cannot be cast
        //to org.apache.spark.sql.catalyst.plans.logical.LogicalPlan
//       val logicalPlan = df.getClass.getDeclaredField("logicalPlan").asInstanceOf[LogicalPlan]
        //
        //try 2: java.lang.NoSuchMethodException: 
        //org.apache.spark.sql.internal.SessionState.executePlan
        //(org.apache.spark.sql.execution.datasources.LogicalRelation)

      
//       val mirror = runtimeMirror(scala.reflect.runtime.universe.getClass.getClassLoader)
//       val instanceMirror = mirror.reflect(df)
//       val field = scala.reflect.runtime.universe.typeOf[DataFrame].decl(TermName("logicalPlan")).asTerm
//       val fieldMirror = instanceMirror.reflectField(field)
//       val logicalPlan = fieldMirror.get.asInstanceOf[LogicalPlan]
      val logicalPlan = getReflectLogicalPlan(df)
    // try 3:
//        val field = classOf[DataFrame].getDeclaredField("logicalPlan")
//        field.setAccessible(true)
//        val logicalPlan = field.get(this).asInstanceOf[LogicalPlan]
      
//       val plan = callPrivate(spark.sessionState,"executePlan",logicalPlan).getClass.getDeclaredField("sparkPlan").asInstanceOf[SparkPlan]
      println("---------------------11111---------")
        val plan = spark.sessionState.executePlan(logicalPlan).sparkPlan
      println("---------------------2222---------")  
        val relation = InMemoryRelation(ser, storageLevel, plan, None, logicalPlan)
//       val relation = InMemoryRelation
      val start = System.currentTimeMillis()
        println("---------------------3333-----!!!----") 
      val cb = relation.cacheBuilder.cachedColumnBuffers.first()
        println("---------------------444---------") 
      val defaWriteTime = System.currentTimeMillis() - start
      verifyFunc(cb)
        println("---------------------555---------") 
      df.unpersist(true)
      println("-----------write cache----cost time:"+defaWriteTime+"-----")  
      defaWriteTime
        
    }
//     CachePerfWriter.appendLine(s"Time taken for writes: $writes")
    val defaWriteTime = writes.sum / 5
    println("-----------write cache----avg cost time:"+defaWriteTime+"-----")  
    val df = query(spark).cache()
    val storageLevel = MEMORY_ONLY
//     val logicalPlan = df.getClass.getDeclaredField("logicalPlan").asInstanceOf[LogicalPlan]
//     val mirror = runtimeMirror(scala.reflect.runtime.universe.getClass.getClassLoader)
//       val instanceMirror = mirror.reflect(df)
//       val field = scala.reflect.runtime.universe.typeOf[DataFrame].decl(TermName("logicalPlan")).asTerm
//       val fieldMirror = instanceMirror.reflectField(field)
//       val logicalPlan = fieldMirror.get.asInstanceOf[LogicalPlan]
    val logicalPlan = getReflectLogicalPlan(df)
//     val plan = callPrivate(spark.sessionState,"executePlan",logicalPlan).getClass.getDeclaredField("sparkPlan").asInstanceOf[SparkPlan]
    val plan = spark.sessionState.executePlan(logicalPlan).sparkPlan
    
    
    val relation = InMemoryRelation(ser, storageLevel, plan, None, logicalPlan)
//     val relation = InMemoryRelation
    
    
    relation.cacheBuilder.cachedColumnBuffers
    val (doExecuteMethod, inMemoryScans) = func(df)
//     CachePerfWriter.appendLine("Reading cache 10 times")
    val reads = for (_ <- 0 until 5) yield {
      val start = System.currentTimeMillis()
      val inMemoryScan = inMemoryScans.head
      println("---------------------AAAAAA---------")
      println(inMemoryScan.getClass)
      val rdd = doExecuteMethod.invoke(inMemoryScan).asInstanceOf[RDD[ColumnarBatch]]
      println("---------------------BBBBB---------")
      if (ser.isInstanceOf[ParquetCachedBatchSerializer] && acc) {
        println("---------------------CCCC111-----!!!----")
        rdd.mapPartitions(iter => CloseableColumnBatchIterator(iter)).count()
        println("---------------------CCCC222---------")
        rdd.foreach {
          println("---------------------CCCC333---------")
          cb => cb.close()
        }
        println("---------------------CCCC4444---------")
      } else {
        println("---------------------DDDDD---------")
        rdd.count()
      }
      
      val defaReadTime = System.currentTimeMillis() - start
      println("-----------read cache-----cost time:"+defaReadTime+"----") 
      defaReadTime

    }
//     CachePerfWriter.appendLine(s"Time taken for reads: $reads")
    val defaReadTime = reads.slice(1, reads.length).sum / 9
    println("-----------read cache----avg cost time:"+defaReadTime+"-----")  

    df.unpersist()
    (defaWriteTime, defaReadTime, reads(0))
//     (defaWriteTime, 1, 1)
  }

// println("aaa")

readWriteCache: (acc: Boolean, spark: org.apache.spark.sql.SparkSession, ser: org.apache.spark.sql.columnar.CachedBatchSerializer, func: org.apache.spark.sql.DataFrame => (java.lang.reflect.Method, Seq[org.apache.spark.sql.execution.SparkPlan]), verifyFunc: org.apache.spark.sql.columnar.CachedBatch => Any, query: org.apache.spark.sql.SparkSession => org.apache.spark.sql.DataFrame)(Long, Long, Long)


In [9]:
import org.apache.spark.sql.execution.columnar.InMemoryTableScanExec


def runDefaInternal[T](
      query: SparkSession => DataFrame,
      acc: Boolean,
      ser: CachedBatchSerializer) = {
    val spark = createSparkSession()
    if (acc) {
      spark.conf.set("spark.rapids.sql.enabled", "true")
    } else {
      spark.conf.set("spark.rapids.sql.enabled", "false")
    }
    println("---------------------"+acc+"---------")
     
  readWriteCache(acc, spark, ser, { df =>
    val doExecuteMethod =
      classOf[InMemoryTableScanExec].getDeclaredMethod("doExecute")
    doExecuteMethod.setAccessible(true)
    val inMemScans = df.queryExecution.executedPlan.collect {
      case m: InMemoryTableScanExec => m
    }
    (doExecuteMethod, inMemScans)
  }, cb =>
    cb match {
      case _: T =>
      case other => throw new IllegalStateException(s"Unexpected cached batch type: ${other.getClass.getName}")
    }, query )
    
  }

<console>:100: warning: abstract type pattern T is unchecked since it is eliminated by erasure
             case _: T =>
                     ^
runDefaInternal: [T](query: org.apache.spark.sql.SparkSession => org.apache.spark.sql.DataFrame, acc: Boolean, ser: org.apache.spark.sql.columnar.CachedBatchSerializer)(Long, Long, Long)


In [10]:
def runPcbsInternal(query: SparkSession => DataFrame, acc: Boolean): (Boolean, Long, Long, Long) = {
    val spark = createSparkSession()
    if (acc) {
      spark.conf.set("spark.rapids.sql.enabled", "true")
    } else {
      spark.conf.set("spark.rapids.sql.enabled", "false")
    }
    println("----------"+acc+"--------------")
    val (pcbsWriteTime, pcbsReadTime, firstRead) =
      readWriteCache(acc, spark, new ParquetCachedBatchSerializer(), { df =>
        val doExecuteMethod =
          classOf[GpuInMemoryTableScanExec].getDeclaredMethod("doExecuteColumnar")
        doExecuteMethod.setAccessible(true)
        val inMemScans = df.queryExecution.executedPlan.collect {
          case g: GpuInMemoryTableScanExec => g
          case m: InMemoryTableScanExec => m
        }
        (doExecuteMethod, inMemScans)
      }, cb =>
        cb match {
          case _: ParquetCachedBatch =>
          case other => throw new IllegalStateException(s"Unexpected cached batch type: ${other.getClass.getName}")
        }, query)

    (acc, pcbsWriteTime, firstRead, pcbsReadTime)
  }

runPcbsInternal: (query: org.apache.spark.sql.SparkSession => org.apache.spark.sql.DataFrame, acc: Boolean)(Boolean, Long, Long, Long)


In [11]:
 
def runPcbs(query: SparkSession => DataFrame): Unit = {
    println("------runPcbsInternal--------")
     runPcbsInternal(query, true)
    println("------runDefaInternal--------")
    val pcbs = new ParquetCachedBatchSerializer()
      runDefaInternal[ParquetCachedBatch](query, false, pcbs)
    println("------recording--------")
 
}

runPcbs: (query: org.apache.spark.sql.SparkSession => org.apache.spark.sql.DataFrame)Unit


In [12]:
val path = "file:///home/yuanli/work/pcbs/part-00070-086b31d9-2f77-4f9b-8d82-a526703fd9d8-c000.snappy.parquet"
try {
//   CachePerfWriter.appendLine(s"Reading file: $path")
  runPcbs(spark => spark.read.parquet(path))
} finally {
//   CachePerfWriter.close()
}

------runPcbsInternal--------
----------true--------------
---------------------11111---------
---------------------2222---------
---------------------3333-----!!!----
---------------------444---------
---------------------555---------
-----------write cache----cost time:695-----
---------------------11111---------
---------------------2222---------
---------------------3333-----!!!----
---------------------444---------
---------------------555---------
-----------write cache----cost time:318-----
---------------------11111---------
---------------------2222---------
---------------------3333-----!!!----
---------------------444---------
---------------------555---------
-----------write cache----cost time:346-----
---------------------11111---------
---------------------2222---------
---------------------3333-----!!!----
---------------------444---------
---------------------555---------
-----------write cache----cost time:295-----
---------------------11111---------
-----------------

path = file:///home/yuanli/work/pcbs/part-00070-086b31d9-2f77-4f9b-8d82-a526703fd9d8-c000.snappy.parquet


-----------read cache-----cost time:165----
-----------read cache----avg cost time:77-----
------recording--------


file:///home/yuanli/work/pcbs/part-00070-086b31d9-2f77-4f9b-8d82-a526703fd9d8-c000.snappy.parquet